In [ ]:
import numpy as np 
import pandas as pd 
import os

In [ ]:
#路径
crf_learn_path = 'CRF++-0.58/crf_learn.exe'
crf_test_path = 'CRF++-0.58/crf_test.exe'
Material_path = '../lab1/199801.txt'
train_file_path = 'train.data'
test_file_path = 'test.data'
template_path = 'template'
model_file_path = 'model'

## 语料处理，生成训练集

In [ ]:
def nofex(word):
    begin = 1 if word[0] == '[' else 0
    mid = word.find('/')
    end = word.find(']')
    if end == -1 : end = len(word) 
    return [word[begin:mid],word[mid+1:end]]

In [ ]:
text = open(Material_path, 'r', encoding="ANSI")
train_data = open(train_file_path, 'w', encoding="UTF-8")

In [ ]:
character_list = []
while(True):
    sentence = text.readline()
    
    if sentence == "\n" or sentence == "":              #一篇文章结束
        if len(character_list) == 0 :
            if sentence == "" : break                   #全文结束 
            continue                                    #多余空行
        for character in reversed(character_list):      #将这篇文章的内容写入 
            if len(character) == 2:
                train_data.write(f"{character[0]}\t{character[1]}\n")  
            else:
                train_data.write("\n")
        character_list = []                             #清空本篇文章的内容
        continue 

    words = sentence.split()[1:]
    words = list(map(nofex, words))

    for word in words :
        for index, character in enumerate(word[0]):
            flag = "B" if index == 0 else "I"
            character_list.append([character, flag])
            if character in ['。','！','……','？']:
                character_list.append(['\n'])
    if character_list[-1][0] != '\n': 
        character_list.append(['\n']) 
  
train_data.close()
text.close()


## 调用CRF程序，生成模型

In [ ]:
abspath = os.path.abspath
os.system(f"cd {abspath('.')} && {abspath(crf_learn_path)} template train.data model > train_output.txt" )

## 测试模型 对比正确度

In [ ]:
os.system(f"cd {abspath('.')} && {abspath(crf_test_path)} -m model test.data > result.txt" )

In [13]:
result_path = 'result.txt'
result = open(result_path, 'r', encoding="UTF-8")

In [14]:
# 以B为正类
total_num, TP, TN, FP, FN= 0, 0, 0, 0, 0

while(True):
    line = result.readline()
    if line == "" : break
    if line == '\n' : continue

    res = line.split()

    total_num += 1
    TP += res[1] == 'B' and res[2] == 'B'
    FN += res[1] == 'B' and res[2] == 'I'
    FP += res[1] == 'I' and res[2] == 'B'
    TN += res[1] == 'I' and res[2] == 'I'

result.close()        

In [17]:
accur = (TP+TN)/total_num
prec = TP/(TP+FP)
recall = TP/(TP+FN)
F1 = 2*prec*recall/(prec+recall)

print(f"accurancy = {accur}")
print(f"prec = {prec}")
print(f"recall = {recall}")
print(f"F1 = {F1}")

accurancy = 0.9711567655095638
prec = 0.9770318021201413
recall = 0.9750629722921914
F1 = 0.9760463943519919
